In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def plot_evaluation_results(file_paths, smooth_length=10):
    """
    Plots the evaluation reward and success rate from multiple NPZ files.
    
    - Computes mean and standard deviation for rewards.
    - Computes mean and percentiles (25th & 75th) for success.
    - Converts time from seconds to hours.
    - Smooths the curves using a moving average.
    - Displays both plots side by side.
    
    Parameters:
        file_paths (list): List of NPZ file paths.
        smooth_length (int): Window size for smoothing the data.
    """

    # Load data
    time_lists, reward_lists, success_lists = [], [], []

    for file_path in file_paths:
        data = np.load(file_path)
        time_lists.append(data["Total_Time_List"])
        reward_lists.append(data["Evaluation_Reward_List"])
        success_lists.append(data["Evaluation_Success_List"])

    # Use the first file's time list as the reference x-axis
    time_reference = time_lists[0]

    # Find the minimum length to align all datasets
    min_length = min(len(t) for t in time_lists)

    # Trim all lists to match the minimum length
    time_trimmed = time_reference[:min_length] / 3600  # Convert seconds to hours
    reward_trimmed = np.array([r[:min_length] for r in reward_lists])
    success_trimmed = np.array([s[:min_length] for s in success_lists])

    # Compute mean and variance for rewards
    reward_mean = np.mean(reward_trimmed, axis=0)
    reward_std = np.std(reward_trimmed, axis=0)

    # Compute mean and percentiles for success rate
    success_mean = np.mean(success_trimmed, axis=0)
    success_25 = np.percentile(success_trimmed, 25, axis=0)
    success_75 = np.percentile(success_trimmed, 75, axis=0)

    # Smoothing function
    def smooth(data, length):
        data = data.copy()
        pad = np.repeat(data[0], length-1)  # Padding with the first value
        pad_data = np.concatenate((pad, data))
        smoothed = np.array([np.mean(pad_data[i:i+length]) for i in range(len(data))])
        return smoothed

    # Apply smoothing
    reward_mean_smooth = smooth(reward_mean, smooth_length)
    reward_std_smooth = smooth(reward_std, smooth_length)

    success_mean_smooth = smooth(success_mean, smooth_length)
    success_25_smooth = smooth(success_25, smooth_length)
    success_75_smooth = smooth(success_75, smooth_length)

    # Plot both figures in one figure (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Plot Evaluation Reward over Time
    axes[0].plot(time_trimmed, reward_mean_smooth, label="Mean Reward", linewidth=3, linestyle="--", color="blue")
    axes[0].fill_between(time_trimmed, reward_mean_smooth + reward_std_smooth, reward_mean_smooth - reward_std_smooth, alpha=0.2, color="blue")
    axes[0].set_xlabel("Time (hours)")
    axes[0].set_ylabel("Reward")
    axes[0].set_title("Smoothed Evaluation Reward")
    axes[0].legend()
    axes[0].grid(True)

    # Plot Evaluation Success over Time with Percentiles
    axes[1].plot(time_trimmed, success_mean_smooth, label="Mean Success", linewidth=3, linestyle="--", color="green")
    axes[1].fill_between(time_trimmed, success_25_smooth, success_75_smooth, alpha=0.2, color="green")
    axes[1].set_xlabel("Time (hours)")
    axes[1].set_ylabel("Success Rate")
    axes[1].set_title("Smoothed Evaluation Success (25th-75th Percentile)")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()


In [11]:
# Example usage
file_paths = [
    "./results/eval_ExpD3_64_128_0.npz",
    "./results/eval_ExpD3_64_128_1.npz",
    "./results/eval_ExpD3_64_128_3.npz",
]
plot_evaluation_results(file_paths, smooth_length=10)

FileNotFoundError: [Errno 2] No such file or directory: './results/eval_ExpD3_64_128_0.npz'